In [ ]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style


%matplotlib inline
style.use('fivethirtyeight')

#dl libraraies
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Concatenate
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.utils import to_categorical
#from tensorflow.keras.utils.np_utils import model_to_dot
from tensorflow.keras.callbacks import ReduceLROnPlateau


from tensorflow.keras.layers import dot
from tensorflow.keras.models import Model


# specifically for deeplearning.
from tensorflow.keras.layers import Dropout, Flatten,Activation,Input,Embedding
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import tensorflow as tf
import random as rn
from IPython.display import SVG

import numpy as np



In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/ratings.csv')

In [ ]:
df.head()

In [ ]:
userlist=df['userId'].unique()

In [ ]:
len(df['userId'].unique())

In [ ]:
userlist

In [ ]:
df['movieId'].unique()

In [ ]:
len(df['movieId'].unique())

In [ ]:
df['userId'].isnull().sum()

In [ ]:
df['rating'].isnull().sum()

In [ ]:
df['movieId'].isnull().sum()

In [ ]:
min_rating=df['rating'].min() # minimum rating
min_rating

In [ ]:
max_rating=df['rating'].max() # maximum rating
max_rating

In [ ]:
df['userId'].value_counts(ascending=True)

In [ ]:
df['movieId'].value_counts(ascending=True)

In [ ]:
df['movieId'].unique()

In [ ]:
#df_mf=df.copy()

In [ ]:
#df_mf

In [ ]:
np.random.seed(0)
test_set=(df.groupby(['userId'])['movieId','rating']
          .apply(pd.Series.sample, n=1).reset_index(level=[0, 1]))

In [ ]:
test_set.keys

In [ ]:
test_index=test_set['level_1'].to_list()
test_index

In [ ]:
df

In [ ]:
df.drop(test_index,axis=0,inplace=True)

In [ ]:
train_set=df.copy()

In [ ]:
train_set

In [ ]:
test_set.drop(['level_1'],axis=1,inplace=True)

In [ ]:
train_set.drop(['timestamp'],axis=1,inplace=True)
train_set

In [ ]:
test_set

#**1. Using SVD**

In [ ]:
!pip install surprise

In [ ]:
from surprise import Dataset,Reader


In [ ]:
reader = Reader(rating_scale=(min_rating, max_rating))

In [ ]:
train_set

!pip install recommenders
from recommenders.datasets.python_splitters import python_random_split
train, test = python_random_split(df, 0.8)

In [ ]:
train_set= Dataset.load_from_df(train_set, reader).build_full_trainset()

In [ ]:
test_set= Dataset.load_from_df(test_set, reader).build_full_trainset()

In [ ]:
test_df=test_set.build_testset()

In [ ]:
!pip install recommenders

In [ ]:
from surprise import SVD
from recommenders.utils.timer import Timer
svd = SVD(random_state=0, n_factors=50,n_epochs=50, verbose=True)

with Timer() as train_time:
    svd.fit(train_set)

print("Took {} seconds for training.".format(train_time.interval))

In [ ]:
#from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k,
                                                    # recall_at_k, get_top_k_items)
#from recommenders.models.surprise.surprise_utils import predict, compute_ranking_predictions

In [ ]:
predictions=svd.test(test_df)

In [ ]:
result = pd.DataFrame(predictions, columns=['user_id', 'movie_id', 'base_event', 'predict_event', 'details'])
result.drop(columns = {'details'}, inplace = True)
result['erro'] = abs(result['base_event'] - result['predict_event'])
result.head()

In [ ]:
!pip install scikit-learn

In [ ]:
import sklearn.metrics
eval_rmse = np.sqrt(sklearn.metrics.mean_squared_error(result['base_event'],result['predict_event']))
eval_rmse

In [ ]:
eval_rsquared = sklearn.metrics.r2_score(result['base_event'],result['predict_event'])
eval_rsquared

#**2. Using NNMF**

In [ ]:
from surprise import NMF
from recommenders.utils.timer import Timer
nmf = NMF(random_state=0, n_factors=50,n_epochs=20, verbose=True)

with Timer() as train_time:
    nmf.fit(train_set)

print("Took {} seconds for training.".format(train_time.interval))

In [ ]:
predictions=nmf.test(test_df)

In [ ]:
nmf['test_rmse']

In [ ]:
eval_rmse = np.sqrt(sklearn.metrics.mean_squared_error(result['base_event'],result['predict_event']))
eval_rsquared = sklearn.metrics.r2_score(result['base_event'],result['predict_event'])
print(round(eval_rmse,4)*100)

In [ ]:
print(round(eval_rsquared,2) )

#**3.1. Using Neural Networks ( SGD optimizer )**

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/ratings.csv')

In [ ]:
df_nn=df.copy()

In [ ]:
# x_train,x_test,y_train,y_test=train_test_split(df[['userId','movieId']],df[['rating']],test_size=0.20,random_state=42)
users = df.userId.unique()
movies = df.movieId.unique()

userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

In [ ]:
userid2idx

In [ ]:
df['userId'] = df['userId'].apply(lambda x: userid2idx[x])
df['movieId'] = df['movieId'].apply(lambda x: movieid2idx[x])

In [ ]:
df

In [ ]:
np.random.seed(0)
test_set=(df.groupby(['userId'])['movieId','rating']
          .apply(pd.Series.sample, n=1).reset_index(level=[0, 1]))

In [ ]:
test_index=test_set['level_1'].to_list()
#test_index

In [ ]:
df.drop(test_index,axis=0,inplace=True)

In [ ]:
train_df=df.copy()

In [ ]:
train_df.drop(['timestamp'],axis=1,inplace=True)

In [ ]:
test_set.drop(['level_1'],axis=1,inplace=True)

In [ ]:
train_df

In [ ]:
test_set

split = np.random.rand(len(df)) < 0.8
train = df[split]
valid = df[~split]
print(train.shape , valid.shape)

In [ ]:
n_latent_factors=50
n_movies=len(df_nn['movieId'].unique())
n_users=len(df_nn['userId'].unique())

In [ ]:
user_input=Input(shape=(1,),name='user_input',dtype='int64')
user_embedding=Embedding(n_users,n_latent_factors,name='user_embedding')(user_input)
user_vec=Flatten(name='FlattenUsers')(user_embedding)
user_vec=Dropout(0.40)(user_vec)

In [ ]:
movie_input=Input(shape=(1,),name='movie_input',dtype='int64')
movie_embedding=Embedding(n_movies,n_latent_factors,name='movie_embedding')(movie_input)
movie_vec=Flatten(name='FlattenMovies')(movie_embedding)
movie_vec=Dropout(0.40)(movie_vec)

In [ ]:
sim=dot([user_vec,movie_vec],name='Simalarity-Dot-Product',axes=1)

In [ ]:
nn_inp=Dense(64,activation='relu')(sim)
nn_inp=Dropout(0.4)(nn_inp)
# nn_inp=BatchNormalization()(nn_inp)
nn_inp=Dense(1,activation='relu')(nn_inp)
nn_model =Model([user_input, movie_input],nn_inp)
nn_model.summary()


In [ ]:
nn_model.compile(optimizer='SGD'#Adam(lr=1e-3)
         ,loss='mse',metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
batch_size=128
epochs=50

In [ ]:
history = nn_model.fit([train_df.userId,train_df.movieId],train_df.rating, batch_size=batch_size,
                              epochs =30, validation_data = ([test_set.userId,test_set.movieId],test_set.rating),
                              verbose = 1)

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
import matplotlib.pyplot as plt
plt.plot(history.history['root_mean_squared_error'] , 'g')
plt.plot(history.history['val_root_mean_squared_error'] , 'b')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.grid(True)
plt.show()

In [ ]:
nn_model.evaluate([test_set.userId,test_set.movieId],test_set.rating)

#**3.2. Using neural Network ( Adam optimizer with .001 leraning rate )**

In [ ]:
nn_adam_model =Model([user_input, movie_input],nn_inp)
nn_adam_model.summary()

In [ ]:

#from tensorflow.keras.metrics import RootMeanSquaredError
nn_adam_model.compile(optimizer=Adam(lr=1e-3)
                 ,loss='mse',metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
history_a = nn_adam_model.fit([train_df.userId,train_df.movieId],train_df.rating, batch_size=256,
                              epochs =20, validation_data = ([test_set.userId,test_set.movieId],test_set.rating),
                              verbose = 1)

In [ ]:
plt.plot(history_a.history['root_mean_squared_error'] , 'g')
plt.plot(history_a.history['val_root_mean_squared_error'] , 'b')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.grid(True)
plt.show()

In [ ]:
nn_adam_model.evaluate([test_set.userId,test_set.movieId],test_set.rating)